In [1]:
%cd ..

/home/david/Desktop/NMEC-Using-DL


/home/david/anaconda3/envs/nmec/lib/python3.9/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


## Setup

In [19]:
import pandas as pd
import eemeter

In [6]:
aust_meter = pd.read_csv('data/program_baseline_daily/opt_prog_mean.csv')
aust_meter['datetime'] = pd.to_datetime(aust_meter['datetime'])

In [8]:
aust_weather = pd.read_csv('data/weather/austin_weather_2018.csv', skiprows = list(range(3)))
aust_weather['time'] = pd.to_datetime(aust_weather['time'])

In [13]:
aust_temp = aust_weather.iloc[:, :2]
temp = pd.Series(data = aust_temp.iloc[:, 1].values, index = aust_temp.iloc[:, 0])

In [17]:
aust_base = aust_meter[['datetime', 'total']]
aust_base.set_index('datetime', inplace=True)

In [18]:
aust_base

,total
datetime,
2018-01-01 01:00:00,988.799667
2018-01-01 02:00:00,995.235455
2018-01-01 03:00:00,994.605636
2018-01-01 04:00:00,991.943364
2018-01-01 05:00:00,990.067909
...,...
2018-12-31 19:00:00,988.602455
2018-12-31 20:00:00,989.393818
2018-12-31 21:00:00,987.984000


In [20]:
meter_data, temperature_data, sample_metadata = (
    eemeter.load_sample("il-electricity-cdd-hdd-hourly")
)


In [29]:
temp.tz_localize('America/Chicago')

NonExistentTimeError: 2018-03-11 02:00:00

In [28]:
temp.info()

<class 'pandas.core.series.Series'>
DatetimeIndex: 8760 entries, 2018-01-01 00:00:00 to 2018-12-31 23:00:00
Freq: H
Series name: None
Non-Null Count  Dtype  
--------------  -----  
8760 non-null   float64
dtypes: float64(1)
memory usage: 136.9 KB


In [22]:
temperature_data.info()

<class 'pandas.core.series.Series'>
DatetimeIndex: 19417 entries, 2015-11-22 06:00:00+00:00 to 2018-02-08 06:00:00+00:00
Freq: H
Series name: tempF
Non-Null Count  Dtype  
--------------  -----  
19417 non-null  float64
dtypes: float64(1)
memory usage: 303.4 KB


In [32]:
import datetime
import pytz

austin_tz = pytz.timezone('America/Chicago')
austin_time = datetime.datetime(2018, 12, 31, 23, 0, 0, tzinfo=austin_tz)


In [40]:
aust_base.index = aust_base.index.tz_localize('America/Chicago', ambiguous=False)

In [45]:
aust_base = aust_base.asfreq('H')

In [49]:
aust_base = aust_base.rename(columns = {'total': 'value'})

## EEmeter

In [99]:
temp = temp.tz_localize('America/Chicago', ambiguous='NaT', nonexistent='NaT')

In [104]:
temp = temp[~temp.index.duplicated(keep='first')]
temp = temp.asfreq('H')


In [105]:
# get meter data suitable for fitting a baseline model
baseline_meter_data, warnings = eemeter.get_baseline_data(
    aust_base, end=austin_time, max_days=365
)

In [106]:
preliminary_design_matrix = (
    eemeter.create_caltrack_hourly_preliminary_design_matrix(
        baseline_meter_data, temp, degc=True
    )
)

In [107]:
segmentation = eemeter.segment_time_series(
    preliminary_design_matrix.index,
    'three_month_weighted',
)

In [108]:
# assign an occupancy status to each hour of the week (0-167)
occupancy_lookup = eemeter.estimate_hour_of_week_occupancy(
    preliminary_design_matrix,
    segmentation=segmentation,
)

/home/david/anaconda3/envs/nmec/lib/python3.9/site-packages/eemeter/features.py:576: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  model_data_with_residuals.groupby(["hour_of_week"])
/home/david/anaconda3/envs/nmec/lib/python3.9/site-packages/eemeter/features.py:576: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  model_data_with_residuals.groupby(["hour_of_week"])
/home/david/anaconda3/envs/nmec/lib/python3.9/site-packages/eemeter/features.py:576: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior 

In [110]:
occupancy_lookup

,dec-jan-feb-weighted,jan-feb-mar-weighted,feb-mar-apr-weighted,mar-apr-may-weighted,apr-may-jun-weighted,may-jun-jul-weighted,jun-jul-aug-weighted,jul-aug-sep-weighted,aug-sep-oct-weighted,sep-oct-nov-weighted,oct-nov-dec-weighted,nov-dec-jan-weighted
0,True,True,True,True,True,False,False,False,False,False,False,True
1,False,True,True,True,True,False,False,False,False,False,False,False
2,False,True,True,True,False,False,False,False,False,True,False,False
3,False,False,True,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...
163,False,False,False,False,False,False,False,False,False,False,False,False
164,False,False,False,False,False,True,True,False,False,False,False,False
165,False,False,False,False,False,True,True,False,False,False,True,False
166,False,False,True,True,True,False,True,False,False,False,False,False


In [111]:
# assign temperatures to bins
occupied_temperature_bins, unoccupied_temperature_bins = eemeter.fit_temperature_bins(
    preliminary_design_matrix,
    segmentation=segmentation,
    occupancy_lookup=occupancy_lookup,
)

/home/david/anaconda3/envs/nmec/lib/python3.9/site-packages/eemeter/features.py:638: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  pd.DataFrame({"temp": temperature_data, "bin": temp_bins})
/home/david/anaconda3/envs/nmec/lib/python3.9/site-packages/eemeter/features.py:638: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  pd.DataFrame({"temp": temperature_data, "bin": temp_bins})
/home/david/anaconda3/envs/nmec/lib/python3.9/site-packages/eemeter/features.py:638: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain cur

In [112]:
# build a design matrix for each monthly segment
segmented_design_matrices = (
    eemeter.create_caltrack_hourly_segmented_design_matrices(
        preliminary_design_matrix,
        segmentation,
        occupancy_lookup,
        occupied_temperature_bins,
        unoccupied_temperature_bins,
    )
)

In [113]:
# build a CalTRACK hourly model
baseline_model = eemeter.fit_caltrack_hourly_model(
    segmented_design_matrices,
    occupancy_lookup,
    occupied_temperature_bins,
    unoccupied_temperature_bins,
)


In [144]:
aust_base.index.dtype

datetime64[ns, America/Chicago]

In [157]:
# use the in-program meter data
in_meter = pd.read_csv('data/program_baseline_daily/in_prog_mean.csv')
in_meter = in_meter[['datetime', 'total']].rename(columns={'total': 'value'})
in_meter['datetime'] = pd.to_datetime(in_meter['datetime'])
in_meter.set_index('datetime', inplace=True)

In [158]:
in_meter.index = in_meter.index.tz_localize('America/Chicago', ambiguous=False)

In [159]:
in_meter = in_meter.asfreq('H')

In [160]:
# get a year of reporting period data
reporting_meter_data, warnings = eemeter.get_reporting_data(
    in_meter, start=datetime.datetime(2018, 1, 1, 0, 0, 0, tzinfo=austin_tz), max_days=365
)

In [161]:
# compute metered savings for the year of the reporting period we've selected
metered_savings_dataframe, error_bands = eemeter.metered_savings(
    baseline_model, reporting_meter_data,
    temperature_data, with_disaggregated=True
)

# total metered savings
total_metered_savings = metered_savings_dataframe.metered_savings.sum()

In [162]:
total_metered_savings

6093.780180329397